In [1]:
import json
import pandas as pd
import jieba
import pickle
import os
from collections import Counter
from pypinyin import pinyin, Style
from deep_translator import GoogleTranslator
import re
from tqdm import tqdm


In [2]:
# 1. Load Data
with open('../data/raw/z_channel_sg_vs_msia.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
full_text = " ".join([d['text'] for d in data])

In [3]:
# 2. Load Custom Malaysian Dictionary
# Ensure you have 'assets/msia_dict.txt' with terms like RTS and CIQ
if os.path.exists('../assets/msia_dict.txt'):
    jieba.load_userdict('../assets/msia_dict.txt')

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 0.827 seconds.
Prefix dict has been built successfully.


In [4]:
# 3. Tokenize & Filter
# Remove stop words and single-character words (grammar particles)
stop_words = {'這個', '這邊', '就是', '一個', '可以', '他們', '因為', '現在', '如果', '一些', '什麼', '所以'}
tokens = [t for t in jieba.lcut(full_text) if len(t) > 1 and t not in stop_words]

In [5]:
# 4. Save for PCA (Notebook 03)
os.makedirs('../data/processed', exist_ok=True)
with open('../data/processed/clean_tokens.pkl', 'wb') as f:
    pickle.dump(tokens, f)

In [6]:
# 5. EDA: Top 10 Phrases
print("Top 10 Phrases:")
print(pd.DataFrame(Counter(tokens).most_common(10), columns=['Word', 'Count']))

Top 10 Phrases:
  Word  Count
0  新加坡     24
1   屋主     23
2   西亞     20
3   自己     18
4   影片     12
5   房間     10
6   分享      9
7   房子      9
8   OK      9
9   我們      9


## For studying

In [7]:
def get_pinyin(text):
    # Style.TONE provides marks like ā, á, ǎ, à
    pinyin_list = pinyin(text, style=Style.TONE)
    return " ".join([item[0] for item in pinyin_list])

def count_mandarin(text):
    # This regex looks for the Unicode range of all Chinese characters
    mandarin_only = re.findall(r'[\u4e00-\u9fff]', text)
    return len(mandarin_only)

def to_seconds(time_str):
    m, s = map(int, time_str.split(':'))
    return m * 60 + s

In [8]:
df = pd.DataFrame(data)

In [9]:
df['mandarin_length'] = df['text'].apply(count_mandarin)
df = df.sort_values(by='mandarin_length', ascending=False)
df = df[df['mandarin_length'] > 0]

In [10]:
# 4. Select the top 30 longest sentences to study
# These are likely to be full explanations or descriptions
study_df = df.head(30).copy()

print(f"⏳ Processing the 30 longest sentences for your study guide...")

⏳ Processing the 30 longest sentences for your study guide...


In [11]:
# Generate Pinyin
study_df['pinyin'] = study_df['text'].apply(get_pinyin)

# Generate English Translation
translator = GoogleTranslator(source='zh-CN', target='en')
study_df['english'] = study_df['text'].apply(lambda x: translator.translate(x))

In [12]:
study_df

,text,start_raw,start_formatted,duration_raw,duration_formatted,mandarin_length,pinyin,english
221,如果你們是還沒有看我房屋糾紛的影片的話,373.233,00:06:13:233,3.033,00:00:03:032,19,rú guǒ nǐ men shì hái méi yǒu kàn wǒ fáng wū j...,If you haven’t watched my video on housing dis...
196,上一次呢我就被一個沒有良心的屋主騙訂金,328.400,00:05:28:399,3.200,00:00:03:200,19,shàng yī cì ne wǒ jiù bèi yī gè méi yǒu liáng ...,"Last time, I was defrauded of a deposit by an ..."
71,可是呢如果是住在馬來西亞自己的房子哦,116.366,00:01:56:365,2.767,00:00:02:766,18,kě shì ne rú guǒ shì zhù zài mǎ lái xī yà zì j...,But what if you live in your own house in Mala...
6,其實呢我從新加坡回到馬來西亞生活呢,8.400,00:00:08:400,2.366,00:00:02:366,17,qí shí ne wǒ cóng xīn jiā pō huí dào mǎ lái xī...,"Actually, I came back to live in Malaysia from..."
267,我要講的是我最後一次租的那間房間啦,451.366,00:07:31:365,3.300,00:00:03:299,17,wǒ yào jiǎng de shì wǒ zuì hòu yī cì zū de nà ...,I'm going to talk about the last room I rented.
114,他們都是允許我們在房間裡面吃東西的,183.700,00:03:03:699,2.600,00:00:02:600,17,tā men dōu shì yǔn xǔ wǒ men zài fáng jiān lǐ ...,They all allow us to eat in the room
52,自己的屋子就絕對不會有這樣子的問題,86.400,00:01:26:400,1.800,00:00:01:800,17,zì jǐ de wū zi jiù jué duì bù huì yǒu zhè yàng...,Your own house will never have such a problem
58,也就是我搬離新加坡租的那間房子哦,96.800,00:01:36:799,3.166,00:00:03:165,16,yě jiù shì wǒ bān lí xīn jiā pō zū de nà jiān ...,That's the house I rented when I moved out of ...
25,這是為了避免一些酸民在那邊耍嘴炮,39.033,00:00:39:033,3.267,00:00:03:266,16,zhè shì wèi le bì miǎn yī xiē suān mín zài nà ...,This is to prevent some sarcastic people from ...
276,我們也會開下午一兩個小時這樣子啦,469.400,00:07:49:399,2.666,00:00:02:665,16,wǒ men yě huì kāi xià wǔ yī liǎng gè xiǎo shí ...,We will also open for an hour or two in the af...


In [13]:
OUTPUT_FILE = '../data/processed/sg_vs_msia_long_sentences_study.csv'
# Adding 'length' to the output so you can see how "big" the sentence is
final_columns = ['start_formatted', 'mandarin_length', 'text', 'pinyin', 'english']
study_df[final_columns].to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

### By Contextual Clip

In [14]:
START_TIME = "00:00"  # 1 minute
END_TIME   = "01:32"  # 2 minutes

In [15]:
start_s = to_seconds(START_TIME)
end_s   = to_seconds(END_TIME)

In [16]:
study_df_context = df[(df['start_raw'] >= start_s) & (df['start_raw'] <= end_s)].copy()
study_df_context = study_df_context.sort_values(by='start_raw')

print(f"🎬 Found {len(study_df_context)} lines between {START_TIME} and {END_TIME}")

🎬 Found 55 lines between 00:00 and 01:32


In [17]:
tqdm.pandas()

study_df_context['pinyin'] = study_df_context['text'].apply(get_pinyin)
study_df_context['english'] = study_df_context['text'].progress_apply(lambda x: translator.translate(x))

  0%|          | 0/55 [00:00<?, ?it/s]

100%|██████████| 55/55 [00:41<00:00,  1.34it/s]


In [18]:
output_context = f'../data/processed/sg_vs_msia_context_{START_TIME.replace(":","-")}.csv'
study_df_context[['start_formatted', 'text', 'pinyin', 'english']].to_csv(output_context, index=False, encoding='utf-8-sig')

print(f"✅ Clip study guide saved: {output_context}")

✅ Clip study guide saved: ../data/processed/sg_vs_msia_context_00-00.csv
